<a href="https://colab.research.google.com/github/Vibha111094/OCR/blob/main/OCR_fun.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytesseract

  Created wheel for pytesseract: filename=pytesseract-0.3.6-py2.py3-none-any.whl size=13629 sha256=289f254c70658b1d5d357ae4626973a69d438faf5a5a0998e8c16852a129c081
  Stored in directory: /root/.cache/pip/wheels/ee/71/72/b98430261d849ae631e283dfc7ccb456a3fb2ed2205714b63f
Successfully built pytesseract


In [ ]:
!pip install opencv-python

In [ ]:
!pip install opencv_wrapper

     |████████████████████████████████| 17.3MB 347kB/s 
     |████████████████████████████████| 25.4MB 1.5MB/s 
  Created wheel for opencv-wrapper: filename=opencv_wrapper-0.2.3-py2.py3-none-any.whl size=18580 sha256=3f9afcec839577bce7922a0e8c94b4348aed4a410e8c3f1fa474b40c64cd1401
  Stored in directory: /root/.cache/pip/wheels/74/fc/22/03106b3bcb2516e69317562a2aa4ceb94b0d5813a448edd928
Successfully built opencv-wrapper
ERROR: umap-learn 0.4.6 has requirement numpy>=1.17, but you'll have numpy 1.16.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled

In [ ]:
!pip install colorama

In [ ]:
import pytesseract
import cv2

In [ ]:
# Import PyTorch Library
import torch
from torch import nn

# Import external libraries
import argparse
import numpy as np
import opencv_wrapper as cvw
from skimage.filters import threshold_local
import json
import random
from string import ascii_uppercase, digits, punctuation
import colorama
import regex

In [ ]:
!which python

/usr/local/bin/python


In [ ]:
!python --version

Python 3.6.9


In [ ]:
image = cv2.imread('invoice.png')

In [ ]:
def get_info(path):
    font     = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.5
    fontColor  = (255,0,0)
    lineType = 1

    #Threshold
    image = cv2.imread(path)

    height,width,channel = image.shape
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    T = threshold_local(gray, 15, offset = 6, method = "gaussian") # generic, mean, median, gaussian
    thresh = (gray > T).astype("uint8") * 255
    thresh = ~thresh

    #Dilation
    kernel =np.ones((1,1), np.uint8)
    ero = cv2.erode(thresh, kernel, iterations= 1)
    img_dilation = cv2.dilate(ero, kernel, iterations=1)

    # Remove noise
    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(img_dilation, None, None, None, 8, cv2.CV_32S)
    sizes = stats[1:, -1] #get CC_STAT_AREA component
    final = np.zeros((labels.shape), np.uint8)
    for i in range(0, nlabels - 1):
        if sizes[i] >= 10:   #filter small dotted regions
            final[labels == i + 1] = 255

    #Find contours
    kern = np.ones((5,15), np.uint8)
    img_dilation = cv2.dilate(final, kern, iterations = 1)
    contours, hierarchy = cv2.findContours(img_dilation, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    # Map contours to bounding rectangles, using bounding_rect property
    rects = map(lambda c: cv2.boundingRect(c), contours)
    # Sort rects by top-left x (rect.x == rect.tl.x)
    sorted_rects = sorted(rects, key =lambda r: r[0])
    sorted_rects = sorted(sorted_rects, key =lambda r: r[1])

    etfo=''
    for rect in sorted_rects:
        x,y,w,h = rect
        if(w<20 or h<20):
            continue
        temp = image[y:y+h, x:x+w]
        temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
        hi = pytesseract.image_to_data(temp, config=r'--psm 6')
        hi = hi.split()
        ind = 22
        while(True):
            if (ind>len(hi)):
                break
            if(int(hi[ind])==-1):
                ind+=11
            else:
                etfo=etfo+hi[ind+1]
                etfo=etfo+" "
                x+=len(hi[ind+1])*20
                ind+=12
        etfo=etfo+'\n'
    return etfo

In [ ]:
!pip install tesseract

     |████████████████████████████████| 45.6MB 158kB/s 
  Created wheel for tesseract: filename=tesseract-0.1.3-cp36-none-any.whl size=45562573 sha256=0adeaf87991c0de16770eed71cf6e2c9fed96787fc65a64e924bbb105a54ebae
  Stored in directory: /root/.cache/pip/wheels/82/1f/d9/24797b123379e4ea9511cf660835468b62dad609634cad2aba
Successfully built tesseract


In [ ]:
class ExtractLSTM(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers=2, bidirectional=True)
        self.linear = nn.Linear(hidden_size * 2, 5)

    def forward(self, inpt):
        embedded = self.embed(inpt)
        feature, _ = self.lstm(embedded)
        oupt = self.linear(feature)
        return oupt

In [ ]:
!sudo apt-get update
!sudo apt-get install libleptonica-dev 
!sudo apt-get install tesseract-ocr tesseract-ocr-dev
!sudo apt-get install libtesseract-dev

Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,697 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu

In [ ]:
!pip install tesseract
!pip install tesseract-ocr

     |████████████████████████████████| 45.6MB 123kB/s 
  Created wheel for tesseract: filename=tesseract-0.1.3-cp36-none-any.whl size=45562573 sha256=b29f4857ff02261a276e5543ce444d4a4b1880cb68875e88c5bc320c99cb6d26
  Stored in directory: /root/.cache/pip/wheels/82/1f/d9/24797b123379e4ea9511cf660835468b62dad609634cad2aba
Successfully built tesseract
  Created wheel for tesseract-ocr: filename=tesseract_ocr-0.0.1-cp36-cp36m-linux_x86_64.whl size=115297 sha256=d5cc8f0eebdc18cd4526755d81704242c70464a575244608f7713ce3f1530d4a
  Stored in directory: /root/.cache/pip/wheels/7d/32/ba/e0852fe25fb388c8108871713a344ec638e231d27f53b742db
Successfully built tesseract-ocr


In [ ]:
!sudo apt-get install tesseract-ocr

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 36 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 0s (28.6 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5

In [ ]:
etfo = get_info('invoice.png')

In [ ]:
etfo

'IPS Shop 27 Plaza ave, Ipsum (000) 23456789012 CASH RECEIPT Adipisc 31.25 Fempor 18.82 Dipsum 2.31 Dreniam 14.30 Vullamco 15.27 Kaboris 4.87 Ecididun 54.50 Magna 75.00 Faliquip 120.50 Total: 459.92 Cash: 370.00 Change: 10.08 THANK YOU FOR SHOPPING \nIPS Shop 27 Plaza ave, Ipsum (000) 23456789012 CASH RECEIPT Adipisc 31.25 Fempor 18.82 Dipsum 2.31 Dreniam 14.30 Vullamco 15.27 Kaboris 4.87 Ecididun 54.50 Magna 75.00 Faliquip 120.50 Total: 459.92 Cash: 370.00 Change: 10.08 THANK YOU FOR SHOPPING \nShop \nIPS \nPlaza \nte \nIpsum \n(000) \n23456789012 \nRECEIPT \nCASH \n31.25 \nAdipisc \n18.862 \nFempor \nDak \nDipsum \n14.30 \nDreniam \n15.27 \nVullamco \n4.87 \nKaboris \nEcididun \n54.50 \n75.00 \nMagna \nFaliquip \n120.50 \nTotal: \n459.92 \nCash: \n370.00 \nChange: \n10.08 \nYOU \nTHANK \nSHOPPING \nFOR \n'

In [ ]:
def pred_to_dict(text, pred, prob):
    res = {"company": ("", 0), "date": ("", 0), "address": ("", 0), "total": ("", 0)}
    keys = list(res.keys())
    seps = [0] + (np.nonzero(np.diff(pred))[0] + 1).tolist() + [len(pred)]
    for i in range(len(seps) - 1):
        pred_class = pred[seps[i]] - 1
        if pred_class == -1:
            continue

        new_key = keys[pred_class]
        new_prob = prob[seps[i] : seps[i + 1]].max()
        if new_prob > res[new_key][1]:
            res[new_key] = (text[seps[i] : seps[i + 1]], new_prob)

    return {k: regex.sub(r"[\t\n]", " ", v[0].strip()) for k, v in res.items()}

 
def test(model):
    model.eval() 
    with torch.no_grad():
            oupt = model(text_tensor)
            prob = torch.nn.functional.softmax(oupt, dim=2)
            prob, pred = torch.max(prob, dim=2)
            prob = prob.squeeze().cpu().numpy()
            pred = pred.squeeze().cpu().numpy()
            real_text = etfo
            result = pred_to_dict(real_text, pred, prob)
            with open("output.json", "w", encoding="utf-8") as json_opened:
                json.dump(result, json_opened, indent=4)
            return result

In [39]:
VOCAB= ascii_uppercase+digits+punctuation+" \t\n"

#Change to CUDA to run using GPU
device = 'cuda'

def get_test_data(etfo):  
    text = etfo
    text_tensor = torch.zeros(len(text), 1, dtype=torch.long)
    text_tensor[:, 0] = torch.LongTensor([VOCAB.find(c) for c in text])
    return text_tensor.to(device)

etfo = get_info('/content/r4.jpeg')
# etfo = get_info('X51005621482.jpeg')
etfo = etfo.upper()
text_tensor = get_test_data(etfo)
temp = []
for i in range(len(text_tensor)):
    if text_tensor[i]>=0 and text_tensor[i]<=70:
        temp.append([int(text_tensor[i])])
text_tensor = torch.LongTensor(temp)

#model initialization
hidden_size = 256
device= torch.device('cpu')
model = ExtractLSTM(len(VOCAB), 16, hidden_size).to(device)
model.load_state_dict(torch.load('/content/model.pth'))
result = test(model)
print(result)

{'company': 'TT PEE EE AE', 'date': 'FRI 04/07/', 'address': 'ALAE  EE,  EEE  YW  A  ,.%  AE  —_—  ET  “PE  AE  A  BAR”  EY  A  WE  TUE  SAGE', 'total': ''}


In [ ]:
/content/ICDAR-2019-SROIE/task3/src/test.py

In [ ]:
etfo

'IPS SHOP 27 PLAZA AVE, IPSUM (000) 23456789012 CASH RECEIPT ADIPISC 31.25 FEMPOR 18.82 DIPSUM 2.31 DRENIAM 14.30 VULLAMCO 15.27 KABORIS 4.87 ECIDIDUN 54.50 MAGNA 75.00 FALIQUIP 120.50 TOTAL: 459.92 CASH: 370.00 CHANGE: 10.08 THANK YOU FOR SHOPPING \nIPS SHOP 27 PLAZA AVE, IPSUM (000) 23456789012 CASH RECEIPT ADIPISC 31.25 FEMPOR 18.82 DIPSUM 2.31 DRENIAM 14.30 VULLAMCO 15.27 KABORIS 4.87 ECIDIDUN 54.50 MAGNA 75.00 FALIQUIP 120.50 TOTAL: 459.92 CASH: 370.00 CHANGE: 10.08 THANK YOU FOR SHOPPING \nSHOP \nIPS \nPLAZA \nTE \nIPSUM \n(000) \n23456789012 \nRECEIPT \nCASH \n31.25 \nADIPISC \n18.862 \nFEMPOR \nDAK \nDIPSUM \n14.30 \nDRENIAM \n15.27 \nVULLAMCO \n4.87 \nKABORIS \nECIDIDUN \n54.50 \n75.00 \nMAGNA \nFALIQUIP \n120.50 \nTOTAL: \n459.92 \nCASH: \n370.00 \nCHANGE: \n10.08 \nYOU \nTHANK \nSHOPPING \nFOR \n'

In [ ]:
model

ExtractLSTM(
  (embed): Embedding(71, 16)
  (lstm): LSTM(16, 256, num_layers=2, bidirectional=True)
  (linear): Linear(in_features=512, out_features=5, bias=True)
)

In [ ]:
!git clone https://github.com/zzzDavid/ICDAR-2019-SROIE

Cloning into 'ICDAR-2019-SROIE'...
remote: Enumerating objects: 94, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 2386 (delta 50), reused 65 (delta 22), pack-reused 2292
Receiving objects: 100% (2386/2386), 278.63 MiB | 25.68 MiB/s, done.
Resolving deltas: 100% (213/213), done.
Checking out files: 100% (1980/1980), done.


In [ ]:
!python /content/ICDAR-2019-SROIE/task3/src/train.py

#0001 | Loss: 1.6259
#0002 | Loss: 1.5316
#0003 | Loss: 1.4297
#0004 | Loss: 1.2627
#0005 | Loss: 1.2578
#0006 | Loss: 1.2494
#0007 | Loss: 1.3070
#0008 | Loss: 1.2499
#0009 | Loss: 1.1691
#0010 | Loss: 1.1963
#0011 | Loss: 1.1676
#0012 | Loss: 1.1626
#0013 | Loss: 1.1177
#0014 | Loss: 1.1089
#0015 | Loss: 1.1304
#0016 | Loss: 1.1061
#0017 | Loss: 1.0683
#0018 | Loss: 1.0492
#0019 | Loss: 1.0408
#0020 | Loss: 0.9769
#0021 | Loss: 0.8429
#0022 | Loss: 0.9167
#0023 | Loss: 0.9411
#0024 | Loss: 0.9112
#0025 | Loss: 0.8677
#0026 | Loss: 0.8099
#0027 | Loss: 0.7039
#0028 | Loss: 0.8260
#0029 | Loss: 0.9794
#0030 | Loss: 0.7049
#0031 | Loss: 0.7673
#0032 | Loss: 0.5654
#0033 | Loss: 0.6884
#0034 | Loss: 0.7551
#0035 | Loss: 0.6197
#0036 | Loss: 0.5376
#0037 | Loss: 0.5320
#0038 | Loss: 0.5669
#0039 | Loss: 0.5402
#0040 | Loss: 0.5982
#0041 | Loss: 0.5009
#0042 | Loss: 0.4410
#0043 | Loss: 0.4334
#0044 | Loss: 0.4491
#0045 | Loss: 0.5517
#0046 | Loss: 0.4621
#0047 | Loss: 0.4222
#0048 | Loss:

In [ ]:
!python /content/ICDAR-2019-SROIE/task3/src/test.py

X00016469670
X00016469671
X51005200931
X51005230605
X51005230616
X51005230621
X51005230648
X51005230657
X51005230659
X51005268275
X51005268408
X51005288570
X51005301666
X51005337867
X51005337877
X51005361906
X51005361908
X51005361912
X51005361923
X51005365187
X51005433518
X51005433543
X51005433548
X51005433556
X51005442322
X51005442334
X51005442343
X51005442366
X51005442375
X51005442382
X51005442388
X51005444040
X51005444041
X51005444044
X51005444046
X51005447841
X51005447842
X51005447844
X51005447851
X51005447859
X51005568855
X51005568866
X51005568885
X51005568887
X51005568889
X51005568890
X51005568892
X51005568894
X51005568895
X51005577191
X51005582745
X51005587261
X51005605287
X51005605295
X51005605332
X51005621482
X51005663274
X51005663300
X51005663307
X51005663309
X51005663310
X51005663323
X51005675099
X51005675103
X51005675104
X51005675914
X51005676534
X51005676535
X51005676537
X51005676542
X51005676548
X51005677327
X51005677333
X51005677336
X51005677337
X510056849111
X5100568494